In [ ]:
####################################################################################
################################# Helper Functions #################################
####################################################################################

def select_random_df(dflist):
    random_noun_df = random.choice(dflist)
    display(random_noun_df)
    return random_noun_df

def get_gender(df):
    possible_genders= ["m","f","n"]
    gender = df.iat[-1,1] # gets the second column of the last row which is where gender is stored
    if gender in possible_genders:
        if gender == "m":
            return "masculine"
        elif gender == "f":
            return "feminine"
        elif gender == "n":
            return "neuter"
        
def select_noun(noun_df,case=None,plurality=None):
    noun=""

    possible_case_rows = ["nominative","genitive","dative","accusative","vocative"]
    possible_plurality_columns = ["singular","dual","plural"]

    if case not in possible_case_rows:
        print("Invalid case")
        return

    if plurality not in possible_plurality_columns:
        print("Invalid plurality")
        return

    noun_row_index_array =get_indices(noun_df,case)[0] # gets an array containing row indexes
    noun_column_index_array=get_indices(noun_df,plurality)[1] # gets an array of column indexes

    if len(noun_row_index_array)==0 or len(noun_row_index_array)==0:
        print("COULDNT FIND WORD, NUMBER OF TRIES")
        return
        
    # if multiple found then select one at random
    noun_row= random.choice(noun_row_index_array)
    noun_column= random.choice(noun_column_index_array)
    
    noun =strip_accents_and_lowercase(
    noun_df.iat[noun_row,noun_column])

    print("Selected Noun: "+noun)
    return noun

def select_verb(verb_df,voice=None,mood=None, plurality=None,person=None,):
    verb=""

    possible_voice_rows =["active","middle"]
    possible_mood_rows =["indicative","subjunctive","optative","imperative"]

    possible_plurality_columns=["singular","dual","plural"]
    possible_person_columns=["first","second","third"]

    if voice not in possible_voice_rows:
        print("Invalid voice")
        return

    if mood not in possible_mood_rows:
        print("Invalid mood")
        return
    
    if plurality not in  possible_plurality_columns:
        print("Invalid plurality")
        return

    if person not in  possible_person_columns:
        print("Invalid person")
        return
    
    print(voice)
    print(mood)
    print(plurality)
    print(person)
    
    voice_row_index_array = get_indices(verb_df,voice)[0]
    mood_row_index_array = get_indices(verb_df,mood)[0]

    plurality_column_index_array = get_indices(verb_df,plurality)[1]
    person_column_index_array = get_indices(verb_df,person)[1]

    verb_row= random.choice(list(set(voice_row_index_array).intersection(mood_row_index_array)))
    verb_column= random.choice(list(set(plurality_column_index_array).intersection(person_column_index_array)))

    verb =strip_accents_and_lowercase(
    verb_df.iat[verb_row,verb_column])

    print("Selected Verb: "+verb)
    return verb

   
def select_adjective(adjective_df,case=None,plurality=None,gender=None):
    adjective=""

    possible_case_rows = ["nominative","genitive","dative","accusative","vocative"]
    possible_plurality_columns=["singular","dual","plural"]
    possible_gender_columns=["masculine","feminine","neuter"]

    if case not in possible_case_rows:
        print("Invalid case")
        return

    if plurality not in possible_plurality_columns:
        print("Invalid mood")
        return
    
    if gender not in  possible_gender_columns:
        print("Invalid plurality")
        return


    case_row_index_array =get_indices(adjective_df,case)[0]
    plurality_column_index_array = get_indices(adjective_df,plurality)[1]
    gender_column_index_array = get_indices(adjective_df,gender)[1]

   

    adjective_row =random.choice(case_row_index_array)
    adjective_column= random.choice(list(set(plurality_column_index_array).intersection(gender_column_index_array)))
    test_adjective_column=list(set(plurality_column_index_array).intersection(gender_column_index_array))

    #print(plurality_column_index_array)
    #print( gender_column_index_array)
    #print(set(plurality_column_index_array).intersection(gender_column_index_array))
    #print(adjective_column)
    
    adjective =strip_accents_and_lowercase(
    adjective_df.iat[adjective_row,adjective_column])

    print("Selected Adjective: "+adjective)
    return adjective

def display_scentnece(scentence):
    for word in scentence:
        display(word[0])

def append_scentence_to_dataset(path,scentence):
    display(path)
    with open(path,"a",encoding="utf-8") as f:
        f.write("[ O -100\n")
        for word in scentence:
            text = word[0].split(" ")
            for i in range(len(text)):
                if(word[2]!=""):
                    if(i==0):
                        f.write(text[i] +" B-ASP "+word[2]+"\n")
                    else:
                        f.write(text[i] +" I-ASP "+word[2]+"\n")
                else:
                    f.write(text[i] +" O "+"-100\n")
        f.write("] O -100\n\n")
    return

#Refer to earlier branches for helper function documentation

In [ ]:
#####################################################################################
################################ Accusative Sentence ################################
#####################################################################################

def generate_accusative_scentence(noun_dfs,verb_dfs,sentiment,case="accusative",plurality="singular",
                                   voice="active",mood="indicative",person="first",randomize=True):
    
    possible_cases = ["nominative","genitive","dative","accusative","vocative"]
    possible_pluralities=["singular","dual","plural"]

    possible_voices =["active","middle"]
    possible_moods =["indicative","subjunctive","optative","imperative"]
    
    possible_persons=["first","second","third"]

   

    if randomize ==True:
        person=random.choice(possible_persons)
        plurality = random.choice(possible_pluralities)
        

    noun_df = select_random_df(noun_dfs)
    verb_df = select_random_df(verb_dfs)
    
    noun =select_noun(noun_df,case,plurality)
    verb =select_verb(verb_df,voice,mood,plurality,person)
    
    output = [
    [verb,"verb",""],
    [noun,"noun",sentiment.capitalize()]
    ]
    #display(output)
    return output


def test_create_accusative_scentence(randomize=True,sentiment="positive",plurality="plural",person="third",tense = "present"):

    matchednouns= match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns")    

    matched_verbs_pos= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs/positive",file_contains=tense)
    matched_verbs_neg= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs/negative",file_contains=tense)

    verbs_pos= get_each_df_from_path_list(matched_verbs_pos,)    
    verbs_neg= get_each_df_from_path_list(matched_verbs_neg)
  
    noun_df_list=get_each_df_from_path_list(matchednouns)

    if debug:display_list(nouns)
    if debug:display_list(verbs_pos_dfs)
    scentence = None
    if (sentiment == "positive"):
        scentence =generate_accusative_scentence(noun_df_list,verbs_pos,sentiment,plurality=plurality,person=person,randomize=randomize)
    elif(sentiment == "negative"):
        scentence =generate_accusative_scentence(noun_df_list,verbs_neg,sentiment,plurality=plurality,person=person,randomize=randomize)
        
     
    display(scentence)
    append_scentence_to_dataset("greek_output/Intermediate/accusative.atepc",scentence)
    return
 
            
        
#####################################################################################
################################ Nominative Sentence ################################
#####################################################################################   

def generate_random_nominative_scentence(noun_dfs,verb_dfs,adjective_dfs,sentiment,case="nominative",plurality="plural",
                                   voice="active",mood="indicative",randomize=True):
    
    
    possible_cases = ["nominative","genitive","dative","accusative","vocative"]
    possible_pluralities=["singular","dual","plural"]

    possible_voices =["active","middle"]
    possible_moods =["indicative","subjunctive","optative","imperative"]
    
    
    possible_persons=["first","second","third"]

    if randomize ==True:
        person=random.choice(possible_persons)
        plurality = random.choice(possible_pluralities)

    noun_df = select_random_df(noun_dfs)
    verb_df = select_random_df(verb_dfs)
    adjective_df = select_random_df(adjective_dfs)

    gender = get_gender(noun_df)

    noun =select_noun(noun_df,case,plurality) #nominative

    verb=select_verb(verb_df,voice,mood,plurality,person)
    adjective= select_adjective(adjective_df,case,plurality,gender)


    
    output = [[noun,"noun",sentiment.capitalize()],[verb,"verb",""],[adjective,"adjective",""]]
    return output
    

def test_create_nominative_scentence(randomize=True,sentiment="positive",plurality = "plural",person="third",tense = "present"):
    matched_nouns= match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns",)
    matched_verbs= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs/",file_contains=tense)
    matched_adj_pos= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")

    nouns=get_each_df_from_path_list(matched_nouns)
    verbs= get_each_df_from_path_list(matched_verbs)
    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)
 
    #display(nouns)
    if debug:display_list(nouns)
    if debug:display_list(verbs)
    if debug:display_list(adjectives_pos)
    
    scentence=[[]]
    
    if (sentiment == "positive"):
        scentence =generate_random_nominative_scentence(nouns,verbs,adjectives_pos,sentiment=sentiment,plurality=plurality,randomize=randomize)
    
    elif(sentiment == "negative"):
        scentence =generate_random_nominative_scentence(nouns,verbs,adjectives_neg,sentiment=sentiment,plurality=plurality,randomize=randomize)
        
    display(scentence)
    append_scentence_to_dataset("greek_output/Intermediate/nominative.atepc",scentence)
    
    return




#####################################################################################
########################### Being and Predicate Sentences ###########################
#####################################################################################

def generate_random_being_and_predicate_sentence(noun_dfs, verb_dfs, adjective_dfs, sentiment, case="accusative", plurality="singular",
                                                 voice="active", mood="indicative", randomize=True, max_retries=5):
    #Retry count added to help determine issues with specific words/cases
    retry_count = 0
    while retry_count < max_retries:
        try:
            if randomize:
                possible_persons = ["first", "second", "third"]
                possible_pluralities = ["singular", "plural"]
                person = random.choice(possible_persons)
                plurality = random.choice(possible_pluralities)

            noun_df = select_random_df(noun_dfs)
            verb_df = select_random_df(verb_dfs)
            adjective_df = select_random_df(adjective_dfs)
            
            gender = get_gender(noun_df)

            noun_one = select_noun(noun_df, "nominative", plurality)
            verb = select_verb(verb_df, voice, mood, plurality, person)
            adjective = select_adjective(adjective_df, "genitive", plurality, gender)
            noun_two = select_noun(noun_df, "genitive", plurality)

            output = [
                [noun_one, "noun one", sentiment.capitalize()],
                [verb, "verb", ""],
                [adjective, "adjective", ""],
                [noun_two, "noun two", sentiment.capitalize()],
            ]
            return output

        except IndexError:
            retry_count += 1
            print(f"Retrying sentence generation... (Attempt {retry_count}/{max_retries})")

    print("Max retries reached. Could not generate a valid sentence.)
    return None

def test_create_being_and_predicate_nominative_sentence(randomize=True, debug=False, sentiment="positive", plurality = "plural",person="third"):
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    matchedverbs = match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs/") 
    matched_adj_pos = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")

    nouns=get_each_df_from_path_list(matchednouns)
    verbs= get_each_df_from_path_list(matchedverbs) 

    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)

    if debug:display_list(nouns)
    if debug:display_list(verbs)
    if debug:display_list(adjectives_pos)

    sentence = [[]]
    if (sentiment == "positive"):
        sentence = generate_random_being_and_predicate_sentence(nouns,verbs,adjectives_pos,sentiment=sentiment,plurality=plurality,randomize=randomize) #Change this
    
    elif(sentiment == "negative"):
        sentence = generate_random_being_and_predicate_sentence(nouns,verbs,adjectives_neg,sentiment=sentiment,plurality=plurality,randomize=randomize) # Change this

    display_scentnece(sentence)
    append_scentence_to_dataset("greek_output/Intermediate/nominative_being_and_predicate.atepc",sentence) # Change this
    return sentence

          
          
#####################################################################################
############################ Subject DO Possesive Action ############################
#####################################################################################
          
def generate_random_sub_do_possessive_action_sentence(noun_dfs, verb_dfs, adjective_dfs, sentiment, case="accusative", plurality="singular",
                                                      voice="active", mood="indicative", randomize=True, max_retries=5):
    retry_count = 0
    while retry_count < max_retries:
        try:
            if randomize:
                possible_persons = ["first", "second", "third"]
                possible_pluralities = ["singular", "plural"]
                person = random.choice(possible_persons)
                plurality = random.choice(possible_pluralities)
            # Select word dataframe
            noun_df = select_random_df(noun_dfs)
            noun_df_2 = select_random_df(noun_dfs)
            noun_df_3 = select_random_df(noun_dfs)
            verb_df = select_random_df(verb_dfs)
            
            # Check for existence of each item in the sentence
            subject = select_noun(noun_df, "nominative", plurality)
            if not subject:
                raise ValueError("Failed to generate subject")

            direct_object = select_noun(noun_df_2, "accusative", plurality)
            if not direct_object:
                raise ValueError("Failed to generate direct object")

            possessive = select_noun(noun_df_3, "genitive", plurality)
            if not possessive:
                raise ValueError("Failed to generate possessive")

            verb = select_verb(verb_df, voice, mood, plurality, person)
            if not verb:
                raise ValueError("Failed to generate verb")

            # Construct the output
            output = [
                [subject, "Subject", sentiment.capitalize()],
                [direct_object, "Direct Object", sentiment.capitalize()],
                [possessive, "Possessive", sentiment.capitalize()],
                [verb, "verb", ""],
            ]
            return output

        except (IndexError, ValueError) as e:
            retry_count += 1
            print(f"Error: {e}. Retrying sentence generation... (Attempt {retry_count}/{max_retries})")

    print("Max retries reached. Could not generate a valid sentence.")
    return None
          
          
def test_create_sub_do_possessive_action_sentence(randomize=True, debug=False, sentiment="positive", plurality = "plural",person="third"):
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    matchedverbs = match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs/") 
    matched_adj_pos = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")

    nouns=get_each_df_from_path_list(matchednouns)
    verbs= get_each_df_from_path_list(matchedverbs) 

    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)

    if debug:display_list(nouns)
    if debug:display_list(verbs)
    if debug:display_list(adjectives_pos)

    sentence = [[]]
    if (sentiment == "positive"):
        sentence = generate_random_sub_do_possessive_action_sentence(nouns,verbs,adjectives_pos,sentiment=sentiment,plurality=plurality,randomize=randomize) #Change this
    
    elif(sentiment == "negative"):
        sentence = generate_random_sub_do_possessive_action_sentence(nouns,verbs,adjectives_neg,sentiment=sentiment,plurality=plurality,randomize=randomize) # Change this

    display_scentnece(sentence)
    append_scentence_to_dataset("greek_output/Intermediate/sub_do_possessive_action.atepc",sentence) 
    return sentence
           

#####################################################################################
########################### Subject DO Possesive Linking ############################
#####################################################################################
          

def generate_random_sub_do_possessive_link_sentence(noun_dfs, verb_dfs, adjective_dfs, sentiment, case="accusative", plurality="singular",
                                                      voice="active", mood="indicative", randomize=True, max_retries=5):
    retry_count = 0
    while retry_count < max_retries:
        try:
            if randomize:
                possible_persons = ["first", "second", "third"]
                possible_pluralities = ["singular", "plural"]
                person = random.choice(possible_persons)
                plurality = random.choice(possible_pluralities)

            noun_df = select_random_df(noun_dfs)
            noun_df_2 = select_random_df(noun_dfs)
            noun_df_3 = select_random_df(noun_dfs)
            verb_df = select_random_df(verb_dfs)

            subject = select_noun(noun_df, "nominative", plurality)
            if not subject:
                raise ValueError("Failed to generate subject")

            direct_object = select_noun(noun_df_2, "accusative", plurality)
            if not direct_object:
                raise ValueError("Failed to generate direct object")

            possessive = select_noun(noun_df_3, "genitive", plurality)
            if not possessive:
                raise ValueError("Failed to generate possessive")

            verb = select_verb(verb_df, voice, mood, plurality, person)
            if not verb:
                raise ValueError("Failed to generate verb")

            # Construct the output
            output = [
                [subject, "Subject", sentiment.capitalize()],
                [direct_object, "Direct Object", sentiment.capitalize()],
                [possessive, "Possessive", sentiment.capitalize()],
                [verb, "verb", ""],
            ]
            return output

        except (IndexError, ValueError) as e:
            retry_count += 1
            print(f"Error: {e}. Retrying sentence generation... (Attempt {retry_count}/{max_retries})")

    print("Max retries reached. Could not generate a valid sentence.")
    return None


          
def test_create_sub_do_possessive_link_sentence(randomize=True, debug=False, sentiment="positive", plurality = "plural",person="third"):
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    matchedverbs = match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs/") 
    matched_adj_pos = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")

    nouns=get_each_df_from_path_list(matchednouns)
    verbs= get_each_df_from_path_list(matchedverbs) 

    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)

    if debug:display_list(nouns)
    if debug:display_list(verbs)
    if debug:display_list(adjectives_pos)

    sentence = [[]]
    if (sentiment == "positive"):
        sentence = generate_random_sub_do_possessive_link_sentence(nouns,verbs,adjectives_pos,sentiment=sentiment,plurality=plurality,randomize=randomize) 
    
    elif(sentiment == "negative"):
        sentence = generate_random_sub_do_possessive_link_sentence(nouns,verbs,adjectives_neg,sentiment=sentiment,plurality=plurality,randomize=randomize) 

    display_scentnece(sentence)
    append_scentence_to_dataset("greek_output/Intermediate/sub_do_possessive_link.atepc",sentence) 
    return sentence
    

In [ ]:
#####################################################################################
########################### Paragraphs - Work in Progress ###########################
#####################################################################################


## Main Paragraph Function
# Set subject
# Set number of sentences
# Randomly choose sentence type in loop
# Create sentences
# Append sentences to list
# Output sentence list


def par_sub_possessive_do_action_sentence(subject_df, sentiment="positive", case="accusative", plurality="singular", person = "first",
                                                     voice="active", mood="indicative", randomize=False, max_retries=15):
    
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    matchedverbs = match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/actionverbs") # adjust this
    nouns=get_each_df_from_path_list(matchednouns)
    verbs=get_each_df_from_path_list(matchedverbs)
    retry_count = 0
    while retry_count < max_retries:
        try:
            if randomize:
                possible_persons = ["first", "second", "third"]
                possible_pluralities = ["singular", "plural"]
                person = random.choice(possible_persons)
                plurality = random.choice(possible_pluralities)

            noun_df = select_random_df(nouns)
            noun_df_2 = select_random_df(nouns)
            verb_df = select_random_df(verbs)
            print("\n\n\n\n\n\n\n", verb_df, "\n\n\n\n\n\n\n\n\n")

            # Validate and generate each part
            subject = select_noun(subject_df, "nominative", plurality)
            if not subject:
                raise ValueError("Failed to generate subject")

            direct_object = select_noun(noun_df, "accusative", plurality)
            if not direct_object:
                raise ValueError("Failed to generate direct object")

            possessive = select_noun(noun_df_2, "genitive", plurality)
            if not possessive:
                raise ValueError("Failed to generate possessive")

            verb = select_verb(verb_df, voice, mood, plurality, person)
            if not verb:
                raise ValueError("Failed to generate verb")

            # Construct the output
            output = [subject,possessive,direct_object,verb]
            return output

        except (IndexError, ValueError) as e:
            retry_count += 1
            print(f"Error: {e}. Retrying sentence generation... (Attempt {retry_count}/{max_retries})")

    # If we reach the maximum retries, raise an error or return None
    print("Max retries reached. Could not generate a valid sentence.")
    return None

def par_being_and_predicate_sentence(subject_df, sentiment="positive", case="accusative", plurality="singular",person = "first",
                                                     voice="active", mood="indicative", randomize=False, max_retries=15):
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    matchedverbs = match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs") # adjust this
    matched_adj_pos = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg = match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")
    nouns=get_each_df_from_path_list(matchednouns)
    verbs=get_each_df_from_path_list(matchedverbs)
    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)
    retry_count = 0
    while retry_count < max_retries:
        try:
            if randomize:
                possible_persons = ["first", "second", "third"]
                possible_pluralities = ["singular", "plural"]
                person = random.choice(possible_persons)
                plurality = random.choice(possible_pluralities)

            noun_df = select_random_df(nouns)
            verb_df = select_random_df(verbs)
            if(sentiment=="positive"):
                adjective_df = select_random_df(adjectives_pos)
            else:
                adjective_df = select_random_df(adjectives_neg)
            
            print(noun_df)
            gender = get_gender(noun_df)
            noun_one = select_noun(subject_df, "nominative", plurality)
            print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n", verb_df, "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
            verb = select_verb(verb_df, voice, mood, plurality, person)
            adjective = select_adjective(adjective_df, "genitive", plurality, gender)
            noun_two = select_noun(noun_df, "genitive", plurality)

            output = [noun_one,verb,adjective,noun_two]
            return output

        except IndexError:
            # If there is an IndexError, increment retry count and try again
            retry_count += 1
            print(f"Retrying sentence generation... (Attempt {retry_count}/{max_retries})")

    # If we reach the maximum retries, raise an error or return None
    print("Max retries reached. Could not generate a valid sentence.")
    return None


def par_nominative_scentence(subject_df,sentiment="positive",case="nominative",plurality="singular",person = "first",
                                   voice="active",mood="indicative",randomize=False):
    matchednouns= match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns")
    matchedverbs= match_words_in_folder("ancient_greek_variations/verb","greek_dictionary/verbs/linkingverbs")
    matched_adj_pos= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg= match_words_in_folder("ancient_greek_variations/adjective","greek_dictionary/adjectives/negative")
    possible_cases = ["nominative","genitive","dative","accusative","vocative"]
    nouns=get_each_df_from_path_list(matchednouns)
    verbs=get_each_df_from_path_list(matchedverbs)
    adjectives_pos=get_each_df_from_path_list(matched_adj_pos)
    adjectives_neg=get_each_df_from_path_list(matched_adj_neg)
    possible_cases = ["nominative","genitive","dative","accusative","vocative"]
    possible_pluralities=["singular","dual","plural"]

    possible_voices =["active","middle"]
    possible_moods =["indicative","subjunctive","optative","imperative"]
    
    
    possible_persons=["first","second","third"]

    if randomize ==True:
        person=random.choice(possible_persons)
        plurality = random.choice(possible_pluralities)

    noun_df = select_random_df(nouns)
    verb_df = select_random_df(verbs)
    if sentiment == "positive":
        adjective_df = select_random_df(adjectives_pos)
    else:
        adjective_df = select_random_df(adjectives_neg)
        
    gender = get_gender(subject_df)

    subject =select_noun(subject_df,case,plurality) #nominative
    
    print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n", verb_df, "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
    verb=select_verb(verb_df,voice,mood,plurality,person)
    print("\n\n\n\n\n\n\n", verb, "\n\n\n\n\n\n\n\n\n")

    adjective= select_adjective(adjective_df,case,plurality,gender)
    
    output = [subject,verb,adjective]
    return output

def paragraph_greek(sentiment="positive", plurality="singular",person = "third", num_sentences=5, randomize = True):
    matchednouns = match_words_in_folder("ancient_greek_variations/noun","greek_dictionary/nouns") 
    nouns=get_each_df_from_path_list(matchednouns)
    subject_df = select_random_df(nouns)
    # Need to make above not a string, but the actual df
    if randomize == True:
        possible_pluralities = ["singular", "plural"]
        person=random.choice(possible_pluralities)
    paragraph = []
    print("\n\n\n\n\n\n",subject_df,"\n\n\n\n\n\n\n")

    sentence_functions = [
        par_being_and_predicate_sentence,
        par_nominative_scentence
    ]
    for i in range(num_sentences):
        sentence_func = random.choice(sentence_functions)
        if (sentiment == "positive"):
            sentence = sentence_func(subject_df, sentiment, plurality=plurality, person=person)
        else: 
            sentence = sentence_func(matchednouns, matchedverbs, matched_adj_neg, sentiment)
        paragraph.append(sentence)
        print("loop complete \n \n \n \n \n \n \n")
    return paragraph
    
